In [1]:
import pandas as pd

In [2]:
DATA_DIR = './Instacart_dataset/'

In [3]:
products = pd.read_csv(f'{DATA_DIR}products.csv')
aisles = pd.read_csv(f'{DATA_DIR}aisles.csv')
departments = pd.read_csv(f'{DATA_DIR}departments.csv')
orders = pd.read_csv(f'{DATA_DIR}orders.csv')
order_products_prior = pd.read_csv(f'{DATA_DIR}order_products__prior.csv')

In [4]:
# --- ステップ1: カテゴリカル特徴量の準備 ---
# productにaisleとdepartmentの情報を結合します。
item_profiles = products.merge(aisles, on='aisle_id', how='left')
item_profiles = item_profiles.merge(departments, on='department_id', how='left')

In [5]:
# --- ステップ2: 購買行動ログから動的特徴量を計算 (その1) ---
# reorder_rate, avg_add_to_cart_order, total_orders を計算します。
prod_features_1 = order_products_prior.groupby('product_id').agg(
    # その商品が再注文された割合 (商品の定番度)
    reorder_rate=('reordered', 'mean'),
    # カートに追加される平均的な順番
    avg_add_to_cart_order=('add_to_cart_order', 'mean'),
    # その商品が注文された総回数 (全体的な人気度)
    total_orders=('order_id', 'count')
).reset_index()

In [6]:
# --- ステップ3: 購買行動ログから動的特徴量を計算 (その2) ---
# unique_users, avg_days_since_prior_order を計算します。
# この計算にはorders.csvの情報(user_id, days_since_prior_order)が必要です。
merged_orders = order_products_prior.merge(orders, on='order_id', how='left')

prod_features_2 = merged_orders.groupby('product_id').agg(
    # その商品を購入したユニークユーザー数
    unique_users=('user_id', 'nunique'),
    # その商品を購入したユーザーの平均注文間隔
    avg_days_since_prior_order=('days_since_prior_order', 'mean')
).reset_index()

In [7]:
# --- ステップ4: 全ての特徴量をproduct_idをキーとして結合 ---
item_profiles = item_profiles.merge(prod_features_1, on='product_id', how='left')
item_profiles = item_profiles.merge(prod_features_2, on='product_id', how='left')

# 購買履歴がない商品（priorセットに一度も登場しない商品）の特徴量はNaNになるため、
# 0で補完します。
item_profiles = item_profiles.fillna(0)

In [8]:
# product_idをデータフレームのインデックスに設定します。
item_profiles.set_index('product_id', inplace=True)

In [9]:
# --- 最終的なアイテムプロファイルデータフレーム (最初の5行) ---
item_profiles.head(5)

,product_name,aisle_id,department_id,aisle,department,reorder_rate,avg_add_to_cart_order,total_orders,unique_users,avg_days_since_prior_order
product_id,,,,,,,,,,
1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,0.613391,5.801836,1852.0,716.0,10.432725
2,All-Seasons Salt,104,13,spices seasonings,pantry,0.133333,9.888889,90.0,78.0,10.482759
3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages,0.732852,6.415162,277.0,74.0,10.565385
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen,0.446809,9.507599,329.0,182.0,14.686207
5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry,0.600000,6.466667,15.0,6.0,12.428571


In [10]:
# --- データフレームの情報 ---
item_profiles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49688 entries, 1 to 49688
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_name                49688 non-null  object 
 1   aisle_id                    49688 non-null  int64  
 2   department_id               49688 non-null  int64  
 3   aisle                       49688 non-null  object 
 4   department                  49688 non-null  object 
 5   reorder_rate                49688 non-null  float64
 6   avg_add_to_cart_order       49688 non-null  float64
 7   total_orders                49688 non-null  float64
 8   unique_users                49688 non-null  float64
 9   avg_days_since_prior_order  49688 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 4.2+ MB


In [11]:
item_profiles.to_csv('tmp_data/01_item_profiles.csv')